In [1]:
!pip install cohere


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cohere: filename=cohere-2.8.0-cp39-cp39-win_amd64.whl size=10935 sha256=9037931e14d47d82f5fc7ecf8d33521184d71776f350a6631d56afdef7f8a463
  Stored in directory: c:\users\amitb\appdata\local\pip\cache\wheels\da\4e\ee\e8ca8b4cfecfae13ee78e9f6cd40c766f09aa9db4604d00c32
Successfully built cohere


In [20]:
!pip install pyyaml


Test non-preprocessed examples

In [2]:
import pandas as pd
data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']


In [3]:
data.head()


,label,body_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,"Nah I don't think he goes to usf, he lives aro..."
2,ham,Even my brother is not like to speak with me. ...
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
4,ham,As per your request 'Melle Melle (Oru Minnamin...


In [4]:
data.shape


(5567, 2)

In [19]:
import cohere
from cohere.classify import Example

# model choices list
models = ['small, medium', 'large']

# get examples list for 5, 25, 125, 625 units
examples_list = []
for i in [5, 25, 125, 625]:
    examples_list.append(data[:i][['body_text', 'label']].apply(
        lambda x: Example(x[0], x[1]), axis=1).to_list())
print(f'example shapes : {len(examples_list[0])}, {len(examples_list[1])}, {len(examples_list[2])}, {len(examples_list[3])}')
print('elements type : ', type(examples_list[0][0]))

# get test inputs
inputs = data[-51:-1]['body_text'].to_list()
print('no of test inputs : ', len(inputs))

# get required preds
req_preds = data[-51:-1]['label'].to_list()
print('no of test labels : ', len(req_preds))


example shapes : 5, 25, 125, 625
elements type :  <class 'cohere.classify.Example'>
no of test inputs :  50
no of test labels :  50


In [21]:
# get api key function
import yaml


def get_api_key(filename):
    try:
        with open(filename, 'r') as config_file:
            config = yaml.load(config_file)
            return config['cohere_ai']['api_key']
    except FileNotFoundError:
        print("'%s' file not found" % filename)


In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

apiKey = get_api_key('auth.yaml')
co = cohere.Client(apiKey)

def evaluate(model, examples, inputs, req_preds):
    """
    evaluation 
    """
    classifications = co.classify(model=model, inputs=inputs, examples=examples)
    print(f"model = {model}; example_size = {len(examples)}; input_size = {len(inputs)}")
    out_preds = [classifications.classifications[i]['prediction'] for i in range(len(inputs))]
    print(f'accuracy = {accuracy_score(req_preds, out_preds)}')
    print(f'precision = {precision_score(req_preds, out_preds)}')
    print(f'recall = {recall_score(req_preds, out_preds)}')
    print(f'f1 = {f1_score(req_preds, out_preds)}')


<ipython-input-21-8c685299af35>:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(config_file)
